In [2]:
from matplotlib import pyplot as plt
from util import non_max_suppression
from modules import Yolo
from PIL import Image
import cv2
import numpy as np
import torchvision.transforms as transforms
import torch
%matplotlib inline

img_path = "Finetune/test/272.png"
label_path = "Finetune/test/272.txt"
num_classes = 4
anchors = [[16, 8], [23, 103], [28, 23], [56, 47], [96, 123], [157, 248]]
using_cuda = torch.cuda.is_available()

img = (transforms.ToTensor())(Image.open(img_path).convert('RGB'))
# read label from text, then write the data into a 50 by 5 pytorch tensor
label = np.loadtxt(label_path).reshape(-1, 5)
label[:, 1], label[:, 3] = label[:, 1] * 512, label[:, 3] * 512
label[:, 2], label[:, 4] = label[:, 2] * 384, label[:, 4] * 384

# model
model = Yolo(anchors, num_classes)
if using_cuda:
    model.cuda()
model.load_state_dict(torch.load("trained_model_params.pt"))
model.eval()

# prediction
input = img.unsqueeze(0)
if using_cuda:
    input = input.cuda()
detections = model(input)
detections = non_max_suppression(detections, 4, conf_thres=0.6, nms_thres=0.3)
# print(detections[0][:, 2] - detections[0][:, 0], detections[0][:, 3] - detections[0][:, 1])

# display prediction
img = np.array(Image.open(img_path).convert('RGB'))
img = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]
for bbox in detections[0]:
    color = int(bbox[-1])
    x1, x2 = max(int(bbox[0].item()), 0), min(int(bbox[2].item()), 512)
    y1, y2 = max(int(bbox[1].item()), 0), min(int(bbox[3].item()), 384)
    cv2.rectangle(img, (x1, y1), (x2, y2), colors[color], 1)

plt.figure()
plt.imshow(img)

plt.show()


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.